In [1]:
import numpy as np
import sys
import os
import vector
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from kinematic_custom import *
from fnn_datagenerator import BatchedFakeParticleDataset_All, flat_output_vars
import pickle

In [2]:
batchsize=320
activation=F.relu
hidden_layers = [256 for i in range(20)]
prefix='try1'

#! copied from try 6. Added kaiming initialization, and made width custom


dfpath='/home/ddemler/HNLclassifier/saved_files/fnn_featregr/all_data/' + prefix + '_losses.csv'
modelsavepath='/home/ddemler/HNLclassifier/saved_files/fnn_featregr/all_data/' + prefix + '_model.pt'

In [3]:
train_batch_size = 320
train_dataset = BatchedFakeParticleDataset_All(batch_size=train_batch_size, length=500_000)

train_dataloader = DataLoader(train_dataset, batch_size=None, shuffle=True, num_workers=8)

val_dataset = BatchedFakeParticleDataset_All(batch_size=1000, length=100_000)
val_dataloader = DataLoader(val_dataset, batch_size=None, shuffle=True, num_workers=8)


input_dim = train_dataset.input_dim
output_dim = train_dataset.output_dim
print(input_dim, output_dim)


15 112


In [4]:
for batch_idx, (input_tensors, output_tensors) in enumerate(train_dataloader):
    i=1
    # print(f"Batch {batch_idx + 1}:")
#     print(f"Input tensors keys: {list(input_tensors.keys())}")
#     print(f"Output tensors keys: {list(output_tensors.keys())}")
#     print(f"Example input tensor ('MET_mass') shape: {input_tensors['MET_mass'].shape}")
#     print(f"Example output tensor ('deltaphi_12') shape: {output_tensors['deltaphi_12'].shape}")
#     # For demonstration purposes, we'll break after printing the first batch
#     break

min max energy_tot before removal 120.10292629921693 1649.1847157249072
min max energy_tot after removalmin max energy_tot before removal  120.10292629921693115.19915103900135  1540.1089896812532473.386545964427

min max energy_tot after removalmin max energy_tot before removalmin max energy_tot before removalmin max energy_tot before removal    122.49536328967265115.98028471227323154.8119962971499125.8562275342073    1630.54921106988742484.8845549743741742.74254236417412044.0643830692725



min max energy_tot after removalmin max energy_tot after removalmin max energy_tot after removal   125.8562275342073115.98028471227323154.8119962971499   1535.0057990217211535.0382350642441595.1299892775264


min max energy_tot before removal 128.6424471972465 2268.877149148733min max energy_tot before removal 
130.7285014151161 1853.891646769493
min max energy_tot after removal 128.6424471972465 1559.8758993097786min max energy_tot after removal
 130.7285014151161 1853.891646769493
min max energy_

KeyboardInterrupt: 

In [5]:
class CustomKinematicNet(nn.Module):
    def __init__(self, input_size, hidden_layers, lenoutput, activation_fn=F.relu):
        super(CustomKinematicNet, self).__init__()
        
        layers = []
        for i in range(len(hidden_layers)):
            # Check if this is a layer where we reintroduce the inputs
            if (i % 3 == 0) and (i != 0):
                in_features = hidden_layers[i-1] + input_size
            else:
                in_features = hidden_layers[i-1] if i > 0 else input_size
            out_features = hidden_layers[i]
            layers.append(nn.Linear(in_features, out_features))
        
        # Adjusting the input size for the final layer
        if len(hidden_layers) % 3 == 0:
            layers.append(nn.Linear(hidden_layers[-1] + input_size, lenoutput))
        else:
            layers.append(nn.Linear(hidden_layers[-1], lenoutput))
        
        self.layers = nn.ModuleList(layers)

        for layer in self.layers:
            # Apply He Initialization
            nn.init.kaiming_normal_(layer.weight, nonlinearity='relu')
            if layer.bias is not None:
                nn.init.constant_(layer.bias, 0)


        self.activation_fn = activation_fn
        
    def forward(self, x):
        inputs = x
        for idx, layer in enumerate(self.layers[:-1]):
            if (idx % 3 == 0) and (idx != 0):
                x = self.activation_fn(layer(torch.cat((x, inputs), dim=-1)))
            else:
                x = self.activation_fn(layer(x))
        
        # Check if the output layer needs the original inputs
        if (len(self.layers) - 1) % 3 == 0:
            return self.layers[-1](torch.cat((x, inputs), dim=-1))
        return self.layers[-1](x)

In [6]:
def custom_loss(y_pred, y_true):
    se_loss = (y_pred - y_true) ** 2
    # print(se_loss.shape)
    num_features = int(output_dim)

    loss_list=[]

    for i in range(num_features -1):

        MSE_meanloss = torch.mean(se_loss[:,i])

        

        loss_list.append(MSE_meanloss.item())
    
    #RMSE for the last feature
    # ((y_pred[:,i]-y_true[:,i])/y_true[:,i])**2
    RMSE_loss = ((y_pred[:,i]-y_true[:,i])/ y_true[:,i])**2
    loss_list.append(torch.mean(RMSE_loss).item())
    se_loss[:,num_features-1] = RMSE_loss
        
    
    full_loss = torch.mean(se_loss)
    return loss_list, full_loss

In [7]:
model = CustomKinematicNet(input_dim, hidden_layers, output_dim, activation_fn=activation)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=30, factor=0.1, verbose=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs=100000

In [8]:
dfcols=flat_output_vars
dfcols = ['train_loss', 'val_loss'] + dfcols
# print(len(dfcols))
if __name__ == "__main__":
    losses_df=pd.DataFrame(columns=dfcols)

    patience=100
    min_loss=np.inf
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch_idx, (data, target) in enumerate(train_dataloader):
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            outputs = model(data)

            loss_list, loss = custom_loss(outputs, target)
            loss.backward()
            
            optimizer.step()
            total_loss += loss.item()

        total_loss /= len(train_dataloader.dataset)
        

        model.eval()
        with torch.no_grad():
            val_loss=0
            for batch_idx, (data, target) in enumerate(val_dataloader):
                data = data.to(device)
                target = target.to(device)
                outputs = model(data)
                loss_list, loss = custom_loss(outputs, target)
                val_loss += loss.item()
            val_loss /= len(val_dataloader.dataset)
            if val_loss < min_loss:
                min_loss=val_loss
                torch.save(model.state_dict(), modelsavepath)
                best_modelchanged=True
                patience=100
            else:
                patience-=1
                if patience==0:
                    print("Early stopping!!", epoch)
                    break
        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_loss)
        new_lr = optimizer.param_groups[0]['lr']

        if old_lr != new_lr and best_modelchanged:
            print("Loading best model due to learning rate decrease...")
            model.load_state_dict(torch.load(modelsavepath))
            best_modelchanged=False

        loss_feats=np.array(loss_list)
        # losses_full=np.array([total_loss]+list(loss_feats))
        losses_full=np.array([total_loss, val_loss]+list(loss_feats))
        
        losses_df.loc[epoch]=losses_full
        # print("loss feats before mean:", loss_feats.shape)
        # loss_feats=np.mean(loss_feats, axis=0)
        # print("loss feats after mean:", loss_feats.shape)
        pd.DataFrame(losses_df).to_csv(dfpath, index=False)
        print("Epoch: {}, Training Loss: {:.4e}, Validation Loss: {:.4e}".format(epoch, total_loss, val_loss))





Epoch: 0, Training Loss: 2.8131e+07, Validation Loss: 5.0177e+06
Epoch: 1, Training Loss: 2.8469e+09, Validation Loss: 1.2340e+08
Epoch: 2, Training Loss: 3.0090e+07, Validation Loss: 2.8071e+09
Epoch: 3, Training Loss: 3.7652e+05, Validation Loss: 3.8414e+04
Epoch: 4, Training Loss: 1.1477e+08, Validation Loss: 2.4489e+07
Epoch: 5, Training Loss: 4.6866e+06, Validation Loss: 6.6001e+06
Epoch: 6, Training Loss: 1.5911e+08, Validation Loss: 2.4106e+07
Epoch: 7, Training Loss: 6.8658e+07, Validation Loss: 1.6208e+07
Epoch: 8, Training Loss: 1.3519e+07, Validation Loss: 4.1410e+06
Epoch: 9, Training Loss: 6.3021e+07, Validation Loss: 5.7153e+06
Epoch: 10, Training Loss: 4.9370e+08, Validation Loss: 3.5357e+05
Epoch: 11, Training Loss: 5.9839e+06, Validation Loss: 4.5591e+09
Epoch: 12, Training Loss: 2.0004e+08, Validation Loss: 1.5087e+06
Epoch: 13, Training Loss: 5.4638e+06, Validation Loss: 9.7874e+06
Epoch: 14, Training Loss: 3.1271e+09, Validation Loss: 9.3577e+06
Epoch: 15, Training 

KeyboardInterrupt: 